# Uniswap V3 - Conversion between Price, sqrtPrice, and tick

- 2020.08.01


In [1]:
from decimal import Decimal
from src.sqrtmath import SqrtPriceMath
import math

smath = SqrtPriceMath()

In [2]:
from src.constants import *

print('min/max_sqrt ratio => tick', smath.getTickAtSqrtRatio(MIN_SQRT_RATIO),
      smath.getTickAtSqrtRatio(MAX_SQRT_RATIO), MIN_TICK, MAX_TICK)
print('min/max_sqrt tick => ratio', smath.getSqrtRatioAtTick(MIN_TICK),
      smath.getSqrtRatioAtTick(MAX_TICK), MIN_SQRT_RATIO, MAX_SQRT_RATIO)

min/max_sqrt ratio => tick -887272 887272 -887272 887272
min/max_sqrt tick => ratio 4295128739 1461446703485210103287273052203988822378723970342 4295128739 1461446703485210103287273052203988822378723970342


## Rules in conversion Price in Python with float

1. To get price, raise power from `tick` (acurate) but use `sqrtPrice` to derive (due to lost precision).

2. To obtain tick, Do not use bit shift but use `/`. Because shift base could result in 0.

We can compare `(sqrtPrice * sqrtPrice) / (2 ** 192)` (a decimal) and `((sqrtPrice * sqrtPrice) >> 192)` (=0) below.

3. Do not use tick to derive price or sqrtPrice unless they happend to be the tick edge.

Order of calculation:

- `Price => sqrtPrice => tick`
- `SqrtPrice => Price`
- Price or sqrtPrice to tick could lose precision from logrithmic operator. So, `Tick x> sqrtPrice`, and `Tick x> Price`.


There are differences due to precision loss when conversion between tick and sqrtPrice.


In [3]:
smath.getSqrtRatioAtTick(194650), smath.getTickAtSqrtRatio(1334901240845780620800419172450304)

(1334871019249706799293742930109995, 194650)

In [4]:
# from sqrtPriceX96 -> tick
sqrtPriceX96 = 1334901240845780620800419172450304
tick = math.log(sqrtPriceX96 * sqrtPriceX96 / (2 ** 192))/math.log(1.0001)
print(int(tick), tick)

# from tick to sqrtPriceX96 (without losing precision)
sqrtpricex96_full = (math.sqrt(1.0001 ** tick) * 2 ** 96)
print(sqrtPriceX96, Decimal(sqrtpricex96_full))

sqrtpricex96_int = (math.sqrt(1.0001 ** int(tick)) * 2 ** 96)
print(sqrtPriceX96, Decimal(sqrtpricex96_int))

print(194650.45281927620 - 194650)

194650 194650.4528192762
1334901240845780620800419172450304 1334901240845780620800419172450304
1334901240845780620800419172450304 1334871019248276165902122954522624
0.4528192761936225


### Example from pools


### pool 1

    // Using DAI/WETH: https://etherscan.io/address/0x60594a405d53811d3bc4766596efd80fd545a270
    fix.dai_weth = {
      token0_decimals: 18,
      token1_decimals: 18,
      liquidity: '2830981547246997099758055',
      sqrtPrice: '1550724133884968571999296281',
      tickSpacing: '60',
    };


In [5]:
liquidity = 2830981547246997099758055
sqrtPrice = 1550724133884968571999296281

tick = smath.getTickAtSqrtRatio(sqrtPrice)
print("SqrtPrice => Tick:", tick)
print("SqrtPrice => Price (Correct with division):", 1 / ((sqrtPrice * sqrtPrice) / (2 ** 192)), '~2610.3')
print("SqrtPrice => Price (Correct with division) (Inverse):", (sqrtPrice * sqrtPrice) / (2 ** 192))
print("SqrtPrice x> Price (Wrong with shift):", (sqrtPrice * sqrtPrice) >> 192)
print("Tick x> Price (Wrong with raising power):", 1.0001 ** tick)

SqrtPrice => Tick: -78677
SqrtPrice => Price (Correct with division): 2610.297910762634 ~2610.3
SqrtPrice => Price (Correct with division) (Inverse): 0.00038309803485527693
SqrtPrice x> Price (Wrong with shift): 0
Tick x> Price (Wrong with raising power): 0.0003830647047208358


### pool 2

    // Using USDC/WETH: https://etherscan.io/address/0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8
    fix.usdc_weth = {
    token0_decimals: 6,
    token1_decimals: 18,
    liquidity: '23187408889601892673',
    sqrtPrice: '1510978141923922864297330642137308',
    tickSpacing: '60',
    };


In [6]:
# pool 2

liquidity = 23187408889601892673
sqrtPrice = 1510978141923922864297330642137308

tick = smath.getTickAtSqrtRatio(sqrtPrice)
print("SqrtPrice => Tick:", tick)
print("SqrtPrice => Price (Correct with division):", 1 / ((sqrtPrice * sqrtPrice) / (2 ** 192)) * 1e12, '~2749.4')
print("SqrtPrice => Price (Correct with division) (Inverse):", (sqrtPrice * sqrtPrice) / (2 ** 192))
print("SqrtPrice x> Price (Wrong with shift):", 1 / ((sqrtPrice * sqrtPrice) >> 192) * 1e12)
print("Tick x> Price (Wrong with raising power):", 1.0001 ** -tick * 1e12)

SqrtPrice => Tick: 197128
SqrtPrice => Price (Correct with division): 2749.4308659156027 ~2749.4
SqrtPrice => Price (Correct with division) (Inverse): 363711636.61428696
SqrtPrice x> Price (Wrong with shift): 2749.430870559225
Tick x> Price (Wrong with raising power): 2749.5889045359063


#### Alternative way of obtaining Tick/SqrtPrice from price ratio

- $tick = log(\text{price ratio}) / log(1.0001)$
- $Price = 1.0001^{tick}$
- $sqrtPrice = 1.0001^{{tick}\over2}$


In [7]:
def _sqrt(_x):
    z = (_x + 1) // 2
    y = _x
    while z < y:
        y = z
        z = (_x // z + z) // 2
    return y


def getSqrtPriceX96(priceA, priceB):
    ratioX192 = (priceA << 192) // priceB
    return _sqrt(ratioX192)


def log2(x, n):
    assert 1 <= x < 2

    result = 0
    for i in range(0, n):
        if x >= 2:
            result += 1 / (2 ** i)
            x /= 2
        x *= x
    return result

In [8]:
priceA = 1 * 10**6
priceB = int(1503.56 / 1 * 10**18)

print('Price:', priceA / priceB)
tick = math.log(priceA / priceB, 100) / math.log(1.0001, 100)
print(f'{tick=}')
print('Tick => Price (without losing precision):', 1.0001 ** tick)
print('Tick => Price (losing precision):', 1.0001 ** int(tick))
print()

sqrtPrice = getSqrtPriceX96(priceA, priceB)
print(f'Price ratio => sqrtPrice: {sqrtPrice}')
sqrtPrice2 = 1.0001 ** (tick / 2) * (2 ** 96)
print(f'tick => sqrtPrice2: {sqrtPrice2}, {Decimal(sqrtPrice2)}')
print()

print('sqrtPrice => Price:', ((sqrtPrice * sqrtPrice) / (2 ** 192)))
print('priceA / priceB - sqrtPrice => Price:', priceA / priceB - ((sqrtPrice * sqrtPrice) / (2 ** 192)))
print()

tick2 = smath.getTickAtSqrtRatio(sqrtPrice)
print('sqrtPrice => tick2:', tick2)
print('Tick x> Price (Wrong with raising power):', 1.0001 ** tick2)

sqrtPrice2 = smath.getSqrtRatioAtTick(tick2)
print(f'Tick x> sqrtPrice2: {sqrtPrice2=}')

Price: 6.65088190694086e-16
tick=-349483.5932592916
Tick => Price (without losing precision): 6.650881906940864e-16
Tick => Price (losing precision): 6.651276468665729e-16

Price ratio => sqrtPrice: 2043239152911629950593
tick => sqrtPrice2: 2.0432391529116305e+21, 2043239152911630467072

sqrtPrice => Price: 6.65088190694086e-16
priceA / priceB - sqrtPrice => Price: 0.0

sqrtPrice => tick2: -349484
Tick x> Price (Wrong with raising power): 6.650611407524977e-16
Tick x> sqrtPrice2: sqrtPrice2=2043197601980699753767


## TickMap


In [9]:
tick = 85176
word_pos = tick >> 8  # or tick // 2**8
bit_pos = tick % 256
print(f"Word {word_pos}, bit {bit_pos}")

Word 332, bit 184


#### Minting and burning

1. When Minting - create liquidity and range

- If $p \in [P_a, P_b]$

$L = { x \over {{1 \over \sqrt{P}} - {1 \over \sqrt{P_b}}} } = { y \over { \sqrt{P} - \sqrt{p_a} } } $

$y / x = { { { \sqrt{P} - \sqrt{p_a} } } \over {{1 \over \sqrt{P}} - {1 \over \sqrt{P_b}}} } $

$y / x = { { { \sqrt{P} - \sqrt{p_a} } } \over {{1 \over \sqrt{P}} - {1 \over \sqrt{P_b}}} } $

- If $p \text{ not } \in [P_a, P_B]$

2. When Burning, use range and liquidity to obtain x and y amount.

- When $p_a > p$, $L = x {{\sqrt{p_a} \sqrt{p_b}} \over { \sqrt{p_b} - \sqrt{p_a}}}$

- When $p > p_b$, $L = {y \over { \sqrt{p_b} - \sqrt{p_a}}}$

- When $p_b > p > p_a$, $x = L {{\sqrt{p_b} - \sqrt{p}} \over {\sqrt{P} \sqrt{p_b}}}$, $y = L (\sqrt{P} - \sqrt{p_a})$


In [10]:
from math import sqrt

tick = 3
tick_a = 0
tick_b = 10

price = 1.0001 ** tick
price_a = 1.0001 ** tick_a
price_b = 1.0001 ** tick_b

y = price

a = (sqrt(price_a) / sqrt(price_b) - 1)
b = (price / sqrt(price_b) + sqrt(price_a))
c = price

L = (-b + sqrt(b*b - 4*a*c)) / 2 / a, (-b - sqrt(b*b - 4*a*c)) / 2 / a

L1 = L[1]
display(L1, price)

(1 + L1 / sqrt(price_b)) * (price + L1 * sqrt(price_a)) - L1 * L1

4001.3001575038193

1.0003000300009999

1.862645149230957e-09

In [11]:
tick = -3
tick_a = 0
tick_b = 10

# token0